<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [27]</a>'.</span>

In [1]:
# + tags=["parameters"]
# declare a list tasks whose products you want to use as inputs
upstream = ['vectorizer_countVec','topic_modeling_disaster_type_final']
random_seed = 42
from tools_rjg import *
import pandas as pd
import re, nltk, spacy, gensim
nltk.download('punkt')
import pyLDAvis.sklearn
import pickle

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
# Parameters
upstream = {
    "vectorizer_countVec": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer_countVec.ipynb",
        "vectorizer": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer_countVec.pkl",
    },
    "topic_modeling_disaster_type_final": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/topic_modeling_disaster_type_final.ipynb",
        "lda_model_earthquake": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/lda_model_earthquake.pkl",
        "lda_model_fire": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/lda_model_fire.pkl",
        "lda_model_flood": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/lda_model_flood.pkl",
        "lda_model_hurricane": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/lda_model_hurricane.pkl",
        "lda_topics_disaster_type": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/lda_topics_disaster_type.csv",
    },
}
product = {
    "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/topic_modeling_disaster_type_results.ipynb"
}


In [3]:
df_all = generate_disaster_type_dataframe(disaster_types = ('earthquake', 'fire', 'flood', 'hurricane')
                                          , dev_train_test= ('dev', 'train', 'test'))

In [4]:
with open("output/vectorizer_countVec.pkl", "rb") as f:
    vectorizer = pickle.load(f)

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
with open("output/lda_model_earthquake.pkl", "rb") as f:
    lda_model_earthquake = pickle.load(f)
with open("output/lda_model_fire.pkl", "rb") as f:
    lda_model_fire = pickle.load(f)
with open("output/lda_model_flood.pkl", "rb") as f:
    lda_model_flood = pickle.load(f)
with open("output/lda_model_hurricane.pkl", "rb") as f:
    lda_model_hurricane = pickle.load(f)

In [6]:
df_all['tweet_text_cleaned'] = df_all['tweet_text'].apply(lambda x: tweet_preprocessing(x))
df_all['lemmatized'] = df_all['tweet_text_cleaned'].apply(lambda x: lemmatize_tweet_text(x, allowed_postags=('NOUN', 'ADJ', 'VERB', 'ADV')))

In [7]:
df_all.sample(100)

,tweet_id,tweet_text,class_label,disaster_type,tweet_text_cleaned,lemmatized
11105,910736396490600448,"Tim Duncan, continuing hurricane relief effort...",rescue_volunteering_or_donation_effort,hurricane,"[duncan, continuing, hurricane, relief, effort...",continue hurricane relief effort wake appear l...
644,797889246593830912,Fair play to all the buildings standing. I cou...,not_humanitarian,earthquake,"[fair, play, buildings, standing, could, barel...",fair play building stand barely stand quake br...
32,728697237153447936,Wildfire official Chad Morrison says between 1...,infrastructure_and_utility_damage,fire,"[wildfire, official, chad, morrison, says, bet...",say structure anzac destroy fire late night
7285,1001579615192846336,Tax Relief May Be Available For Maryland Prope...,infrastructure_and_utility_damage,flood,"[relief, available, maryland, property, owners...",relief available property owner impact heavy r...
4011,1061324057033887744,Research CALIFORNIA fires +DIRECTED ENERGY WEA...,not_humanitarian,fire,"[research, california, fires, directed, energy...",fire direct energy weapon agenda wildland proj...
...,...,...,...,...,...,...
27173,1168199019170082816,"Have we mentioned, stay vigilant? Be prepared,...",other_relevant_information,hurricane,"[have, mentioned, stay, vigilant, prepared, ci...",mention stay vigilant prepared city parkland e...
661,1032244136575782913,Where can I donate clothes&amp;foodstuffs 4 #K...,rescue_volunteering_or_donation_effort,flood,"[where, donate, clothes, foodstuffs, kodagu, f...",donate clothe foodstuff kodagu flood affect lo...
4433,1032155571514953728,#WATCH: Indo-Tibetan Border Police (ITBP) pers...,rescue_volunteering_or_donation_effort,flood,"[watch, indo, tibetan, border, police, itbp, p...",watch police itbp personnel transport patient ...
2523,907320602335334400,Our devs are working on technology to gather s...,rescue_volunteering_or_donation_effort,hurricane,"[devs, working, technology, gather, supply, vo...",devs work technology gather supply volunteer n...


In [8]:
earthquake_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='earthquake']['lemmatized']))
fire_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='fire']['lemmatized']))
flood_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='flood']['lemmatized']))
hurricane_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='hurricane']['lemmatized']))

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['donate'] not in stop_words.
  warnings.warn(


## Performance Stats

In [9]:
# Log Likelyhood: Higher the better
print("Log Likelihood Earthquake: ", lda_model_earthquake.score(earthquake_text_vectorized))
print("Log Likelihood Fire: ", lda_model_fire.score(fire_text_vectorized))
print("Log Likelihood Flood: ", lda_model_flood.score(flood_text_vectorized))
print("Log Likelihood Hurricane: ", lda_model_hurricane.score(hurricane_text_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity Earthquake: ", lda_model_earthquake.perplexity(earthquake_text_vectorized))
print("Perplexity Fire: ", lda_model_fire.perplexity(fire_text_vectorized))
print("Perplexity Flood: ", lda_model_flood.perplexity(flood_text_vectorized))
print("Perplexity Hurricane: ", lda_model_hurricane.perplexity(hurricane_text_vectorized))

Log Likelihood Earthquake:  -488653.45987242664


Log Likelihood Fire:  -862991.27309673


Log Likelihood Flood:  -1017564.9037416612


Log Likelihood Hurricane:  -3262678.1450290033


Perplexity Earthquake:  1515.1718974502674


Perplexity Fire:  1864.339285829549


Perplexity Flood:  1798.312954109303


Perplexity Hurricane:  2185.718504207312


In [10]:
get_dominant_topics(lda_model_earthquake,earthquake_text_vectorized)

,Topic0,Topic1,dominant_topic
Doc0,0.780000,0.220000,0
Doc1,0.900000,0.100000,0
Doc2,0.050000,0.950000,1
Doc3,0.440000,0.560000,1
Doc4,0.430000,0.570000,1
Doc5,0.930000,0.070000,0
Doc6,0.060000,0.940000,1
Doc7,0.910000,0.090000,0
Doc8,0.960000,0.040000,0
Doc9,0.870000,0.130000,0


In [11]:
get_dominant_topics(lda_model_fire,fire_text_vectorized)

,Topic0,Topic1,Topic2,Topic3,dominant_topic
Doc0,0.130000,0.620000,0.130000,0.130000,1
Doc1,0.020000,0.020000,0.930000,0.020000,2
Doc2,0.040000,0.610000,0.040000,0.320000,1
Doc3,0.840000,0.050000,0.060000,0.050000,0
Doc4,0.040000,0.480000,0.430000,0.040000,1
Doc5,0.030000,0.910000,0.030000,0.030000,1
Doc6,0.030000,0.810000,0.140000,0.030000,1
Doc7,0.030000,0.030000,0.920000,0.030000,2
Doc8,0.310000,0.560000,0.060000,0.060000,1
Doc9,0.040000,0.040000,0.460000,0.460000,2


In [12]:
get_dominant_topics(lda_model_flood,flood_text_vectorized)

,Topic0,Topic1,Topic2,dominant_topic
Doc0,0.180000,0.770000,0.050000,1
Doc1,0.050000,0.780000,0.170000,1
Doc2,0.380000,0.540000,0.080000,1
Doc3,0.060000,0.890000,0.060000,1
Doc4,0.090000,0.770000,0.140000,1
Doc5,0.020000,0.300000,0.680000,2
Doc6,0.030000,0.940000,0.030000,1
Doc7,0.870000,0.110000,0.020000,0
Doc8,0.860000,0.030000,0.120000,0
Doc9,0.030000,0.940000,0.030000,1


In [13]:
get_dominant_topics(lda_model_hurricane, hurricane_text_vectorized)

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.020000,0.190000,0.690000,0.090000,0.020000,2
Doc1,0.020000,0.020000,0.020000,0.710000,0.250000,3
Doc2,0.170000,0.090000,0.020000,0.710000,0.020000,3
Doc3,0.020000,0.020000,0.750000,0.110000,0.110000,2
Doc4,0.460000,0.240000,0.270000,0.020000,0.020000,0
Doc5,0.470000,0.010000,0.310000,0.090000,0.110000,0
Doc6,0.020000,0.350000,0.020000,0.600000,0.020000,3
Doc7,0.020000,0.020000,0.020000,0.020000,0.920000,4
Doc8,0.010000,0.570000,0.080000,0.010000,0.320000,1
Doc9,0.190000,0.090000,0.660000,0.050000,0.010000,2


## Get Topic Distribution

In [14]:
get_topic_distribution(lda_model_earthquake, vectorizer)

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aameen,abaco,abandon,abandon pet,abfire,ability,able,able donate,able help,abortion,...,yike,yorker,young,young child,youth,yymfire,zealand,zimbabwean,zone,zone evacuate
Topic0,16.246544,0.5,0.938747,0.5,0.5,1.662618,0.624759,0.501778,0.530943,0.5,...,3.816277,0.508962,0.847534,0.503202,1.973659,0.5,37.803243,0.5,22.984212,2.399711
Topic1,0.560295,0.5,0.506942,0.5,0.5,1.352203,26.868264,1.373845,7.249722,0.5,...,1.500512,1.258413,22.201422,1.606142,1.239694,0.5,0.577323,0.5,0.612041,0.519063


In [15]:
get_topic_distribution(lda_model_fire, vectorizer)

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aameen,abaco,abandon,abandon pet,abfire,ability,able,able donate,able help,abortion,...,yike,yorker,young,young child,youth,yymfire,zealand,zimbabwean,zone,zone evacuate
Topic0,0.25,0.25,0.284904,0.250304,0.256143,0.266406,0.255386,0.256373,0.250033,0.250208,...,0.25,0.25,0.262424,0.263879,0.250265,0.254651,0.25,0.25,44.405086,0.25
Topic1,0.25,0.25,21.388881,0.311233,0.254219,0.253929,0.274535,4.703197,0.257331,0.278534,...,0.25,0.25,0.275972,0.260282,0.250385,11.527771,0.25,0.25,0.272364,0.25
Topic2,0.25,0.25,0.304140,0.250305,0.257205,3.835102,0.261526,0.260674,0.252680,1.151167,...,0.25,0.25,0.288779,0.267948,0.250283,0.266247,0.25,0.25,0.304135,0.25
Topic3,0.25,0.25,0.301005,3.125786,14.024657,0.266785,42.033378,0.264797,1.947330,0.735181,...,0.25,0.25,34.541918,8.777497,5.196342,0.261983,0.25,0.25,0.270690,0.25


In [16]:
get_topic_distribution(lda_model_flood, vectorizer)

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aameen,abaco,abandon,abandon pet,abfire,ability,able,able donate,able help,abortion,...,yike,yorker,young,young child,youth,yymfire,zealand,zimbabwean,zone,zone evacuate
Topic0,0.333806,0.333333,5.008241,0.688236,0.333333,0.509957,34.128366,0.360233,8.167390,0.333333,...,0.333333,0.362379,0.506311,3.630370,0.480628,0.333333,0.333333,0.333333,0.642682,0.333333
Topic1,0.333694,0.333333,0.380639,0.372659,0.333333,0.438830,19.179508,0.769730,0.359055,0.333333,...,0.333333,0.375276,56.140387,0.454526,25.490031,0.333333,0.333333,0.333333,7.854235,0.333333
Topic2,0.755636,0.333333,0.519816,0.333806,0.333333,5.827860,17.901032,0.334792,0.365608,0.333333,...,0.333333,1.231153,0.441722,0.396678,0.354652,0.333333,0.333333,0.333333,8.269187,0.333333


In [17]:
get_topic_distribution(lda_model_hurricane, vectorizer)

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aameen,abaco,abandon,abandon pet,abfire,ability,able,able donate,able help,abortion,...,yike,yorker,young,young child,youth,yymfire,zealand,zimbabwean,zone,zone evacuate
Topic0,0.2,68.213737,0.201798,0.200191,0.2,0.205166,0.206094,0.200136,0.200155,0.201054,...,18.950327,0.201714,60.995829,0.370710,0.201731,0.2,0.2,0.206319,0.203144,0.208299
Topic1,0.2,0.207933,0.202156,0.202843,0.2,0.213691,0.206537,0.200095,0.200719,0.204407,...,0.203839,0.201206,0.207833,2.632738,0.202448,0.2,0.2,0.207154,367.485403,10.262516
Topic2,0.2,0.204936,0.202333,0.202419,0.2,18.432910,0.208123,0.200341,0.202614,11.053798,...,0.209440,0.201488,0.207498,0.385004,0.202645,0.2,0.2,0.205911,0.203519,0.211460
Topic3,0.2,0.202393,35.103785,15.782635,0.2,0.201927,180.066132,0.204195,0.206401,0.203196,...,0.206272,0.200181,0.212401,0.498254,0.203587,0.2,0.2,0.206046,0.204091,0.200465
Topic4,0.2,0.205192,0.203604,0.203136,0.2,0.202887,0.210010,3.623760,20.338363,0.211711,...,0.200829,10.096288,0.218836,0.915644,26.918241,0.2,0.2,120.307269,0.201617,0.203084


## Show Top N Topics

In [18]:
show_topics(vectorizer, fitted_lda_model=lda_model_earthquake, n_words=20)

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
_Topic 0,pray,prayer,safe,injure,thought,damage,affect,kill,magnitude,dead,family,stay,quake,least,hope,thought prayer,report,keep,stay safe,area
_Topic 1,help,donate,death,victim,rescue,need,toll,death toll,relief,rise,team,effort,support,thank,donation,emergency,toll rise,affect,quake,city


In [19]:
show_topics(vectorizer, fitted_lda_model=lda_model_fire, n_words=20)

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
_Topic 0,death,toll,death toll,camp,dead,state,miss,kill,deadly,least,rise,news,still,toll rise,history,search,continue,break,official,burn
_Topic 1,relief,support,help,make,donation,effort,donate,climate,change,camp,victim,climate change,come,cause,first,fund,start,disaster,evacuation,also
_Topic 2,home,lose,destroy,burn,family,firefighter,many,pray,life,year,take,rain,time,die,devastating,area,give,town,go,send
_Topic 3,help,camp,victim,donate,blame,live,need,animal,evacuee,good,money,work,raise,evacuate,want,displace,tweet,know,resident,show


In [20]:
show_topics(vectorizer, fitted_lda_model=lda_model_flood, n_words=20)

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
_Topic 0,help,need,relief,rescue,food,camp,family,affect,pray,stand,operation,rebuild,life,relief camp,team,need help,item,save,contact,friend
_Topic 1,relief,donate,victim,fund,help,relief fund,donation,affect,work,material,support,thank,effort,send,relief material,contribute,donate relief,money,come,medical
_Topic 2,flooding,state,damage,disaster,year,rain,say,flash,miss,news,assistance,bad,prayer,time,death,thought,take,cause,national,go


In [21]:
show_topics(vectorizer, fitted_lda_model=lda_model_hurricane, n_words=20)

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
_Topic 0,storm,damage,destroy,wind,flooding,go,death,coast,category,report,toll,kill,rain,tropical,death toll,least,leave,bring,dead,break
_Topic 1,evacuation,say,life,destruction,disaster,provide,order,update,county,community,mandatory,move,open,mandatory evacuation,road,follow,reach,span,approach,evacuation order
_Topic 2,damage,pray,home,cause,time,power,still,area,evacuate,much,hope,resident,take,look,first,many,good,make,include,island
_Topic 3,emergency,safe,supply,rescue,stay,news,food,team,state,cyclone,response,shelter,assist,recovery,stay safe,prepare,ready,effort,change,affected
_Topic 4,help,relief,need,donate,victim,prayer,affect,cyclone,thought,donation,family,effort,thank,thought prayer,volunteer,support,keep,relief effort,give,fund


## Extract most representative sentence for each topic

In [22]:
pd.options.display.max_colwidth = 1000

In [23]:
earthquake_top_sentence = extract_most_representative_sentence(lda_model_earthquake, earthquake_text_vectorized, df_all[df_all['disaster_type']=='earthquake'])
for topic in range(lda_model_earthquake.n_components):
    print('----------------------')
    print(f'Topic_{topic}', earthquake_top_sentence[f'Topic_{topic}'][1]['tweet_text'])

----------------------
Topic_0 5111    The strong #Earthquake of 5.8 magnitude has brought disasters to different parts of #Pakistan which claimed at least 21 lives and 500 left injured so far. Our heartfelt condolences to the families of the victims and prayers for the speedy recovery of those wounded! #Saverspk
Name: tweet_text, dtype: object
----------------------
Topic_1 1667    ὓ8Rescue Emergency has been declared in Azad Kashmir &amp; District Jhelum. ὓ8 Swift response by Pak Army Aviation, Medical Corps, and Soldiers on ground. ὓ8 National &amp; Provincial Disaster Management Authorities are into it. ὓ8 Volunteers are mobilized on ground. #earthquake
Name: tweet_text, dtype: object


In [24]:
fire_top_sentence = extract_most_representative_sentence(lda_model_fire, fire_text_vectorized, df_all[df_all['disaster_type']=='fire'])
for topic in range(lda_model_fire.n_components):
    print('----------------------')
    print(f'Topic_{topic}', fire_top_sentence[f'Topic_{topic}'][1]['tweet_text'])

----------------------
Topic_0 5581    Revised death toll in Paradise, Californias Camp Fire the states deadliest now stands at 76 with 1,276 still missing and the death toll expected to rise dramatically.
Name: tweet_text, dtype: object
----------------------
Topic_1 1294    CoBank has committed $350,000 to support relief efforts in the Northern California communities impacted by the #Camp Fire. The bank is partnering with customers and California Farm Credit associations to support those impacted by the fire.
Name: tweet_text, dtype: object
----------------------
Topic_2 1920    May all my friends and family be safe and my thoughts and prayers to out to the ones were hurt and lost loved ones during these tragic fires in California.
Name: tweet_text, dtype: object
----------------------
Topic_3 2123    Educators and school personnel helping students cope with the #California wildfires can use NCTSN tip sheet Helping Youth after Community Trauma: Tips for Educators for suggestions on h

In [25]:
flood_top_sentence = extract_most_representative_sentence(lda_model_flood, flood_text_vectorized, df_all[df_all['disaster_type']=='flood'])
for topic in range(lda_model_flood.n_components):
    print('----------------------')
    print(f'Topic_{topic}', flood_top_sentence[f'Topic_{topic}'][1]['tweet_text'])

----------------------
Topic_0 1131    #KeralaFloods #KeralaFloodRelief ὄ9Indian Coast Guards Rescue &amp; Relief helicopter carried 1000 kg of general medicines,food packets,drinking water,sanitary pads,baby diapers,bed sheets toothpaste etc..&amp; dropped them remotest &amp; inaccessible flooded areas
Name: tweet_text, dtype: object
----------------------
Topic_1 3888    The Faculty, Residents &amp; Staff @JipmerNephro contributed Rice, Sugar &amp; Pulses worth INR 21000/- for #KeralaFloodRelief - in addition to their personal contributions, volunteering for relief work &amp; joining #JIPMER fund collection drive from staff salary contributions
Name: tweet_text, dtype: object
----------------------
Topic_2 1873    Nebraska Severe Weather Awareness Week continues today with the topic of Tornadoes. Do you know what to do if a Tornado Warning is issued for your area or you see a tornado? NOW is the time to make a plan! Use the TEST Tornado Warning today to practice your plan. #newx #swa

In [26]:
hurricane_top_sentence = extract_most_representative_sentence(lda_model_hurricane, hurricane_text_vectorized, df_all[df_all['disaster_type']=='hurricane'])
for topic in range(lda_model_hurricane.n_components):
    print('----------------------')
    print(f'Topic_{topic}', hurricane_top_sentence[f'Topic_{topic}'][1]['tweet_text'])

----------------------
Topic_0 3867    Although downgraded to a tropical depression with sustained winds of 35 mph, #Florence continues to produce heavy rain..Rivers continue to rise with widespread flooding.. At least 17 people have been killed @FOXLA
Name: tweet_text, dtype: object
----------------------
Topic_1 11475    Mandatory evacuation issued for Ocracoke Island ahead of Hurricane Maria  #MANDATORY #EVACUATION Hyde County issued
Name: tweet_text, dtype: object
----------------------
Topic_2 25605    Dorian will not do that much damage Dorian will not do that much damage Dorian will not do that much damage Dorian will not do that much damage Dorian will not do that much damage Dorian will not do that much damage Dorian will not do that much damage
Name: tweet_text, dtype: object
----------------------
Topic_3 177    #LACoFD’s California Task Force 2 (CA-TF2) will be deploying to North Carolina in anticipation of #HurricaneFlorence. TF2’s team will consist of 16 members trained i

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [27]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_earthquake, earthquake_text_vectorized, vectorizer, mds='tsne')
panel

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/

ValueError: perplexity must be less than n_samples